**Generate NDHM**

In [ ]:
import sys
import os
import laspy
import numpy as np
from osgeo import gdal
import time
from progressbar import ProgressBar

In [ ]:
# Input files, parameters
las_fn = './in2018_29651885_12_las13.las'
dtm_fn = './in2018_29651885_12_dtm.img'
ndhm_fn = './in2018_29651885_12_ndhm_5_ft.tif'
out_format = 'GTiff'
scale_factor = 2.0

In [ ]:
# Open LiDAR file
las = laspy.file.File(las_fn, mode='r')
points = np.vstack((las.x, las.y, las.z)).transpose()
num_points = points.shape[0]

# Open DTM file
dtm = gdal.Open(dtm_fn)

In [ ]:
# Check data info
dtm_array = dtm.ReadAsArray()
nrow = dtm_array.shape[0]
ncol = dtm_array.shape[1]
print('nrow and ncol:', nrow, ncol)
gt = dtm.GetGeoTransform()
print('Geotransform:', gt)
print('Cellsize(x,y):', gt[1], gt[5])

**[Affine transform](http://resources.esri.com/help/9.3/arcgisengine/java/gp_toolref/coverage_tools/how_transform_coverage_works.htm)**


Xgeo = gt[0]   +   (col \* gt[1])

Ygeo = gt[3]   +   (row \* gt[5])

In [ ]:
ul_x = gt[0]
ul_y = gt[3]
cs_x = gt[1]
cs_y = gt[5]

In [ ]:
dtm.GetProjection()

In [ ]:
nrow_out = int(nrow/scale_factor)
ncol_out = int(ncol/scale_factor)
cs_x_out = cs_x * scale_factor
cs_y_out = cs_y * scale_factor
gt_out = [ul_x, cs_x_out, gt[2], ul_y, gt[4], cs_y_out]

In [ ]:
# Close las file
las.close()

In [ ]:
start = time.time()

# Initialize DSM and NDHM array
dsm = np.zeros((nrow_out, ncol_out), dtype=np.float32)
ndhm = np.zeros((nrow_out, ncol_out), dtype=np.float32)

pbar = ProgressBar()
for i in pbar(range(num_points)):
    # Get LiDAR point
    p_x = points[i,0]
    p_y = points[i,1]
    p_z = points[i,2]
    
    # Get row, col index in image of the point
    col = int((p_x - ul_x) / cs_x_out)
    row = int((p_y - ul_y) / cs_y_out)
    col_dtm = int(col * scale_factor)
    row_dtm = int(row * scale_factor)
    
    # Check points outside DTM boundary
    if col < 0 or col >= ncol_out:
        print('x out of dtm boundary', p_x)
        continue
    if row < 0 or row >= nrow_out:
        print('y out of dtm boundary', p_y)
        continue
    
    # Update DSM, NDHM
    if dsm[row, col] < p_z:
        dsm[row, col] = p_z
        ndhm[row, col] = p_z - dtm_array[row_dtm, col_dtm]
        
# Save output
driver = gdal.GetDriverByName(out_format)
ndhm_ds = driver.Create(ndhm_fn, xsize=ncol_out, ysize=nrow_out, bands=1, eType=gdal.GDT_Float32)
ndhm_ds.SetProjection(dtm.GetProjection())
ndhm_ds.SetGeoTransform(gt_out)
ndhm_ds.GetRasterBand(1).WriteArray(ndhm)
ndhm_ds = None

# Print elapsed time
end = time.time()
print('Elapsed time:', int(end - start))